In [ ]:
#import and clean data
#importing packages
import warnings
warnings.simplefilter("ignore")
import numpy as np
from google.colab import files
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
import collections
import nltk
nltk.download('vader_lexicon')
from google.colab import drive
import os
import json
import re
import math
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt

#connect to google drive
drive.mount('/content/drive/')
path="/content/drive/My Drive/devided_dataset_v2/"
os.chdir(path)
os.listdir(path)
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/data.csv

#read data from google drive
#train = pd.read_json('/content/drive/My Drive/devided_dataset_v2/CDs_and_Vinyl/train/review_training.json')
#train = pd.read_json('/content/drive/My Drive/devided_dataset_v2/CDs_and_Vinyl/test2/review_test.json')
awesometrain = pd.read_json('/content/drive/My Drive/devided_dataset_v2/CDs_and_Vinyl/train/product_training.json')

parsedData = pd.read_json('/content/drive/My Drive/devided_dataset_v2/vaderHistogramStandardized.json')
parsedTest = pd.read_json('/content/drive/My Drive/devided_dataset_v2/vaderHistogramStandardizedTest.json')

#test = pd.read_json('/content/drive/My Drive/devided_dataset_v2/CDs_and_Vinyl/test1/review_test.json')
#awesometest = pd.read_json('/content/drive/My Drive/devided_dataset_v2/CDs_and_Vinyl/test1/product_test.json')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


MessageError: ignored

In [ ]:
trainData = pd.DataFrame(parsedData)
#trainData = trainData.drop(['reviews', 'summaries', 'asin'], axis=1)
testData = pd.DataFrame(parsedTest)
#testData = testData.drop(['reviews', 'summaries', 'asin'], axis=1)
awesometrain = awesometrain.drop("asin", axis=1)

In [ ]:
df = pd.DataFrame(train)
df['unixReviewTime'] = pd.to_numeric(df['unixReviewTime'])
df = df.sort_values('asin')

#test.columns = ['asin', 'reviewerID', 'unixReviewTime', 'vote', 'verified', 'reviewTime', 'style', 'reviewerName', 'reviewText', 'summary', 'image']
#test = test.drop(labels=0, axis=0)
#dftest = pd.DataFrame(test)
#dftest['unixReviewTime'] = pd.to_numeric(dftest['unixReviewTime'])
#dftest = dftest.sort_values('asin')

In [ ]:
awesometrain = awesometrain.drop("asin", axis=1)
awesometrain

,awesomeness
0,1
1,0
2,0
3,1
4,0
...,...
73077,1
73078,1
73079,0
73080,1


In [ ]:
def sentimentalAnalysis(df):
  # initialize varaibles
  sol = np.zeros([2, df.shape[0]])
  sentanal = SentimentIntensityAnalyzer()

  # loop through all reviews
  for i in range(df.shape[0]):

    # if text is empty, then sentimental analysis=0
    if not df['reviewText'].iloc[i]:
      sol[0][i] = 0;
    else:
      #otherwise, calculate Sentimental analysis values
      SAValue = sentanal.polarity_scores(df['reviewText'].iloc[i])
      sol[0][i] = SAValue["compound"]

    #do sent anal on summary text
    if not df['summary'].iloc[i]:
      sol[1][i] = 0;
    else:
      #otherwise, calculate Sentimental analysis values
      SAValue = sentanal.polarity_scores(df['summary'].iloc[i])
      sol[1][i] = SAValue["compound"]
  return sol

In [ ]:
def uniqueItems(df):
  #initialize Variables
  uniqueItems = 1
  curritem = df['asin'].iloc[0]

  #loop through all rows
  for i in range(df.shape[0]):

    #check if the review is unique
    if df['asin'].iloc[i] != curritem:
      curritem = df['asin'].iloc[i]
      uniqueItems = uniqueItems + 1
      #end of if statement

  return uniqueItems

#unique = uniqueItems(trainData)

In [ ]:
#parse data
def parseRow(votes, unixReview, reviewCount, nones, verified, compound,
             compoundsumm, reviewlens, summarylens, reviews, summaries, asin):
      row = np.zeros(57, dtype=object)

      #Percent of Nones final Tally
      row[0] = nones / reviewCount

      #Max Votes
      row[1] = np.max(votes)

      #Average Vote
      row[2] = np.average(votes)

      #Standard deviation votes
      row[3] = np.std(votes)

      #Percent of verified
      row[4] = np.sum(verified)/reviewCount

      #Minimum Review Time
      row[5] = np.min(unixReview)

      #Maximum Review Time
      row[6] = np.max(unixReview)

      #Average review time
      row[7] = np.average(unixReview)

      #Standard Deviation of review time
      row[8] = np.std(unixReview)

      #Number of reviews
      row[9] = reviewCount

      #sentimental analysis
      row[10] = np.average(compound)
      row[11] = np.std(compound)

      #next 18 are sentiment histogram
      #next 18 after that are votes counter histogram
      for i in range(len(compound)):
        #split compound into histogram
        index = 12
        if compound[i] != 0:
          index = index + math.ceil((compound[i]*4)+4)
        if verified[i] == 0:
          index = index + 9
        row[index] = row[index] + 1

        #count the votes
        index = index + 18
        row[index] = row[index] + votes[i]

      #turn the compound sentanal into percentage
      for i in range(12, 30):
        row[i] = row[i]/len(compound)

      row[48] = np.average(compoundsumm)
      row[49] = np.std(compoundsumm)

      #length of review and summary
      row[50] = np.average(reviewlens)
      row[51] = np.average(summarylens)
      row[52] = np.std(reviewlens)
      row[53] = np.std(summarylens)

      #review and summary raw text
      row[54] = ''
      row[55] = ''
      for i in range(len(reviews)):
        if reviews[i] != None:
          row[54] = row[54] + reviews[i] + ' '
        if summaries[i] != None:
          row[55] = row[55] + summaries[i] + ' '

      #asin
      row[56] = asin

      return row

def parseData(unique, SA, df):
  #add rows and columns to aggregated data
  newData = pd.DataFrame(columns=["Percent of Nones", "Max Votes", "Average Votes",
                                  "Standard Deviation Votes", "Percent of Verified",
                                  "Minimum Review Time", "Maximum Review Time",
                                  "Average Review Time", "Stand Deviation of Review Time",
                                  "Number of Reviews", "compound average", "compound SD",
                                  "% 0", "% -1 to -.75", "% -.75 to -.5", "% -.5 to -.25", "% -.25 to 0",
                                  "% 0 to .25", "% .25 to .5", "% .5 to .75", "%.75 to 1",
                                  "%uv 0", "%uv -1 to -.75", "%uv -.75 to -.5", "%uv -.5 to -.25", "%uv -.25 to 0",
                                  "%uv 0 to .25", "%uv .25 to .5", "%uv .5 to .75", "%uv .75 to 1",
                                  "#votes 0", "#votes -1 to -.75", "#votes -.75 to -.5", "#votes -.5 to -.25", "#votes -.25 to 0",
                                  "#votes 0 to .25", "#votes .25 to .5", "#votes .5 to .75", "#votes .75 to 1",
                                  "#votes uv 0", "#votes uv -1 to -.75", "#votes uv -.75 to -.5", "#votes uv -.5 to -.25", "#votes uv -.25 to 0",
                                  "#votes uv 0 to .25", "#votes uv .25 to .5", "#votes uv .5 to .75", "#votes uv .75 to 1",
                                  "compound average summ", "compound SD summ",
                                  "avg length of review", "avg length of summary",
                                  "std length of review", "std length of summary",
                                  "reviews", "summaries", "asin"],
                         index = range(unique))
  #initialize variables
  uniqueItems = 0
  itemcount = 0
  curritem = df['asin'].iloc[0]
  votes = []
  unixReview = []
  reviewCount = 0
  nones = 0
  verified = []

  #sentimental anaylsis variables
  compound = []
  compoundsumm = []
  reviewlens = []
  summarylens = []
  reviews = []
  summaries = []

  #loop through every row in data
  for i in range(df.shape[0]):
    if (df['asin'].iloc[i] != curritem):
      itemcount=itemcount+1

      #append array into panda dataframe
      newData.loc[uniqueItems] = parseRow(votes, unixReview, reviewCount, nones,
                                          verified, compound, compoundsumm, reviewlens, summarylens,
                                          reviews, summaries, df['asin'].iloc[i-1])

      #reset the value of the variables
      nones = 0
      reviewCount = 0
      votes = []
      unixReview = []
      verified = []
      compound = []
      compoundsumm = []
      reviewlens = []
      summarylens = []
      reviews = []
      summaries = []
      uniqueItems = uniqueItems + 1
      # end of if statement
    curritem = df['asin'].iloc[i]
    reviewCount = reviewCount + 1

    #Percent of Nones counting number of nums
    if df['vote'].iloc[i] == None:
      nones += 1
      votes.append(0)
    else:
      #Votes
      numVotes = int(re.sub(",", "", df['vote'].iloc[i]))
      votes.append(numVotes)

    #Count verified
    if df['verified'].iloc[i] == True:
      verified.append(1)
    else:
      verified.append(0)

    #append unixReviewTime
    unixReview.append(int(df['unixReviewTime'].iloc[i]))

    #sentimental analysis
    compound.append(SA[0][i])
    compoundsumm.append(SA[1][i])

    #length of review and summary
    if df['reviewText'].iloc[i] != None:
      reviewlens.append(len(df['reviewText'].iloc[i]))
    else:
      reviewlens.append(0)

    if df['summary'].iloc[i] != None:
      summarylens.append(len(df['summary'].iloc[i]))
    else:
      summarylens.append(0)

    #review and summary raw text
    reviews.append(df['reviewText'].iloc[i])
    summaries.append(df['summary'].iloc[i])

    #end of for loop

  # parse last rows data
  newData.loc[unique - 1] = parseRow(votes, unixReview, reviewCount, nones,
                                     verified, compound, compoundsumm, reviewlens, summarylens,
                                     reviews, summaries, df['asin'].iloc[unique-1])

  return newData

def standardize(df):
  result = df.copy()
  for feature_name in df.columns:
    if feature_name not in ["reviews", "summaries", "asin"] and feature_name[0] != '%':
      #convert column to numeric
      result[feature_name] = pd.to_numeric(df[feature_name])

      #this line is the calculation
      if result[feature_name].std() != 0 and result[feature_name].max() != 0:

        result[feature_name] = (result[feature_name]-result[feature_name].mean()) / result[feature_name].std()
        #make 0 lowest value
        result[feature_name] = result[feature_name] + abs(result[feature_name].min())
        #standardize to 0-1
        result[feature_name] = result[feature_name]/result[feature_name].max()
        print("standardized column ", feature_name)
  return result

dfsmall = df[:2000]
unique = uniqueItems(df)
trainDataSA = sentimentalAnalysis(df)
trainData = parseData(unique, trainDataSA, df)

trainDataStandardized = standardize(trainData)

print(trainData.iloc[0])

In [ ]:
print(trainDataStandardized.iloc[1])

In [ ]:
def standardize(df):
  result = df.copy()
  for feature_name in df.columns:
    #convert column to numeric
    result[feature_name] = pd.to_numeric(df[feature_name])

    #standardize data
    max_value = result[feature_name].max()
    min_value = result[feature_name].min()
    #this line is the calculation
    result[feature_name] = (result[feature_name]-result[feature_name].mean()) / result[feature_name].std()
    result[feature_name] = result[feature_name] + result[feature_name].min()
  return result

trainData = trainData.iloc[:,:30]
trainData = standardize(trainData)
trainData.head(3)

In [ ]:
#Download data
with open('vaderHistogramStandardizedTest.json', 'w') as f:
  f.write(trainDataStandardized.to_json())
files.download('vaderHistogramStandardizedTest.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, chi2
#for i in range(10):
tfidf = TfidfVectorizer(stop_words ='english')
X_train, X_test, y_train, y_test = train_test_split(trainData, awesometrain, test_size=0.25)
tog = X_train['reviews'] + X_train['summaries']
togtest = X_test['reviews'] + X_test['summaries']

transf = tfidf.fit_transform(tog.fillna(""))
transf = pd.DataFrame.sparse.from_spmatrix(transf)
transftest = tfidf.transform(togtest.fillna(""))
tramsftest = pd.DataFrame.sparse.from_spmatrix(transftest)

selection = SelectPercentile(chi2, percentile=12)
trainDataTFIDF = selection.fit_transform(transf, y_train)
trainDataTFIDFTest = selection.transform(transftest)

trainDataTFIDF = pd.DataFrame.sparse.from_spmatrix(trainDataTFIDF)
trainDataTFIDFTest = pd.DataFrame.sparse.from_spmatrix(trainDataTFIDFTest)

In [ ]:
import scipy.sparse
from scipy.sparse import hstack

X_trainsparse = scipy.sparse.csr_matrix(X_train.iloc[:,:54].values)
trainDataTFIDF = hstack((X_trainsparse, trainDataTFIDF))

In [ ]:
print(X_trainsparse)

In [ ]:
import scipy.sparse
from scipy.sparse import hstack
#dtype = pd.SparseDtype(float, fill_value=0)
#col = X_train.iloc[:,:54].astype(dtype)


for column in X_train.iloc[:,:54].columns:
  trainDataTFIDF = trainDataTFIDF.drop([column], axis=1)
  trainDataTFIDF[column] = X_train[column]
  print('appended', column)

for rfw in range(0, 5):
  for dtc in range(0, 5):
    for threshold in range(.4, .65, .05):
      sum = rfw+dtc
      predweight = (rfw*predsrfw + dtc*predsdtc)/sum
      if predweight > threshold:
        class 1
      else
        class 2

In [ ]:
#trainDataTFIDF = trainDataTFIDF.fillna(0)
print(X_train.iloc[2])
print(trainDataTFIDF.iloc[2])

In [ ]:
col = X_test.iloc[:,:54].astype(dtype)
train = hstack((col, trainDataTFIDFtest))

In [ ]:
#feature selection function

from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

# takes in a classifier, training data, and the classes and returns the
# ranked best indices of the training data
def featureSelection(clf, trainData, awesometrain):
  #initialize variables
  columns = trainData.columns
  scores = np.zeros(trainData.shape[1])


  #baseline KNN
  baseLine = np.average(cross_val_score(clf, trainData, awesometrain, scoring='f1',
                                        cv=10))
  # loop through all columns
  for i in range(trainData.shape[1]):

    # calculate score of column
    scores[i] = -1 * (np.average(cross_val_score(clf, trainData.iloc[:, trainData.columns != columns[i]],
                           awesometrain, scoring='f1', cv=10)) / baseLine - 1)
    #print("Column Name: %s, f1-score: %.3f" % (columns[i], scores[i]))
    # end of for

    #Find most and least impactful features
  Indices = np.argsort(scores)
  '''
  print("Top 5 impactful features:")
  for i in range(5):
    print("Feature: %s: %f" % (columns[Indices[-1*i - 1]], scores[Indices[-1*i - 1]]))

  print("Bottom 5 impactful features:")
  for i in range(5):
    print("Feature: %s: %f" % (columns[Indices[i]], scores[Indices[i]]))

  # plot results
  plt.figure(figsize=(20,10))

  # creating the bar plot
  plt.bar(range(len(columns)), scores, color ='maroon',
          width = 1)
  plt.xlabel("Feature index")
  plt.ylabel("Percent worse")
  plt.title("Feature Selection")
  plt.show()
  '''
  return Indices

In [ ]:
# recursive Feature Select
# takes in a classifer, the dataframe and a datafram of the class
# outputs the best f1-score from recursing through features greedily
from sklearn.model_selection import train_test_split

def RFS(clf, trainData, awesomeTrain, visited={}, startF1 = 0):

  #initialize variables
  columns = trainData.columns
  bestF1 = 0

  #baseline KNN
  baseLine = np.average(cross_val_score(clf, trainData, awesometrain, scoring='f1',
                                        cv=10))
  # loop through all columns
  for i in range(trainData.shape[1]):
    # if we remove a column and it is empty, then we have gone to far
    if trainData.iloc[:, trainData.columns != columns[i]].empty: return [0, visited]

    #initialize variable
    listx = columns[columns != columns[i]]

    # calculate score of column
    if visited.get(frozenset(listx)):
      return [visited.get(frozenset(listx)), visited]
    else:
      x_train, x_test, y_train, y_test = train_test_split(trainData.iloc[:, trainData.columns != columns[i]],
                                                          awesometrain, test_size = 0.1)
      clf.fit(x_train, y_train)
      F1 = f1_score(y_test, clf.predict(x_test))
      score = -1 * (F1 / baseLine - 1)
      visited[frozenset(listx)] = F1

    bestF1 = max(bestF1, F1, startF1)
    print(bestF1)

    # if score is negative, throw out the column
    if score < 0:
      temp, visited = RFS(clf, trainData.iloc[:, trainData.columns != columns[i]], awesometrain, visited=visited, startF1=bestF1)
      bestF1 = max(bestF1, temp)
    else:
      bestF1 = max(bestF1, F1)

    #end function
  return [bestF1, visited]

In [ ]:
#PCA feature reduction
trainDataCP = trainData.copy()

pca = PCA(n_components=10)
trainDataCP = pca.fit_transform(trainDataCP)
trainDataCP = pd.DataFrame(trainDataCP)

In [ ]:
#Decision Trees
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

absBestOutput = ""
absBestScore = 0

for n in range(3, 30, 3):
  bestOutput = ""
  bestScore = 0
  trainDataCP = trainData.copy()
  pca = PCA(n_components=n)
  trainDataCP = pca.fit_transform(trainDataCP)
  trainDataCP = pd.DataFrame(trainDataCP)
  for c in ['entropy', 'gini', 'log_loss']:
    for s in ['best', 'random']:
      for i in range(1, math.floor(n/2)+5):
        clf = DecisionTreeClassifier(criterion=c, splitter=s, max_depth = i)

        scores = cross_val_score(clf, trainDataCP, awesometrain, scoring='f1_weighted', cv=10)
        output = 'pca folds: %d, criterion: %s, splitter: %s, max_depth: %d, F1: %f, F1std: %f' % (n, c, s, i, np.average(scores), np.std(scores))
        if np.average(scores)-np.std(scores) > bestScore:
          bestOutput = output
          bestScore = np.average(scores)-np.std(scores)
          print('new best:', output)
        print(output)
  print('The best option for decision tree is: %s' % (bestOutput))
  if(bestScore > absBestScore):
    absBestOutput = bestOutput
    absBestScore = bestScore
    print('new best in general, ', absBestOutput)

print('The best option in general is %s' % (absBestOutput))



In [ ]:
bestOutput = ""
bestScore = 0
n = 0
for c in ['entropy', 'gini', 'log_loss']:
  for s in ['best', 'random']:
    for i in range(3, 16):
      clf = DecisionTreeClassifier(criterion=c, splitter=s, max_depth = i)

      scores = cross_val_score(clf, trainDataCP, awesometrain, scoring='f1_weighted', cv=5)
      output = 'pca folds: %d, criterion: %s, splitter: %s, max_depth: %d, F1: %f, F1std: %f' % (n, c, s, i, np.average(scores), np.std(scores))
      if np.average(scores)-np.std(scores) > bestScore:
        bestOutput = output
        bestScore = np.average(scores)-np.std(scores)
      print(output)
print('The best option for decision tree is: %s' % (bestOutput))

pca folds: 0, criterion: entropy, splitter: best, max_depth: 3, F1: 0.565771, F1std: 0.004125
pca folds: 0, criterion: entropy, splitter: best, max_depth: 4, F1: 0.572591, F1std: 0.005799
pca folds: 0, criterion: entropy, splitter: best, max_depth: 5, F1: 0.575635, F1std: 0.003551
pca folds: 0, criterion: entropy, splitter: best, max_depth: 6, F1: 0.576188, F1std: 0.004706
pca folds: 0, criterion: entropy, splitter: best, max_depth: 7, F1: 0.580451, F1std: 0.007199
pca folds: 0, criterion: entropy, splitter: best, max_depth: 8, F1: 0.584081, F1std: 0.004517
pca folds: 0, criterion: entropy, splitter: best, max_depth: 9, F1: 0.586092, F1std: 0.003093
pca folds: 0, criterion: entropy, splitter: best, max_depth: 10, F1: 0.584341, F1std: 0.004663
pca folds: 0, criterion: entropy, splitter: best, max_depth: 11, F1: 0.582012, F1std: 0.004210
pca folds: 0, criterion: entropy, splitter: best, max_depth: 12, F1: 0.575474, F1std: 0.005069
pca folds: 0, criterion: entropy, splitter: best, max_dep

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
n = 12
c = 'log_loss'
s = 'best'
i = 6
averages = []
for x in range(5):
  trainDataCP = trainData.copy()
  pca = PCA(n_components=n)
  trainDataCP = pca.fit_transform(trainDataCP)
  trainDataCP = pd.DataFrame(trainDataCP)
  clf = DecisionTreeClassifier(criterion=c, splitter=s, max_depth = i)

  scores = cross_val_score(clf, trainDataCP, awesometrain, scoring='f1', cv=10)
  averages.append(np.average(scores))
  print('pca folds: %d, criterion: %s, splitter: %s, max_depth: %d, F1: %f' % (n, c, s, i, np.average(scores)))

print('average score: %f' % np.average(averages))

pca folds: 12, criterion: log_loss, splitter: best, max_depth: 6, F1: 0.636327
pca folds: 12, criterion: log_loss, splitter: best, max_depth: 6, F1: 0.635835
pca folds: 12, criterion: log_loss, splitter: best, max_depth: 6, F1: 0.635261
pca folds: 12, criterion: log_loss, splitter: best, max_depth: 6, F1: 0.635307
pca folds: 12, criterion: log_loss, splitter: best, max_depth: 6, F1: 0.636601
average score: 0.635866


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
bestOutput = ""
bestScore = 0
n = 0
for c in ['entropy', 'gini', 'log_loss']:
  for i in range(7, 15):
    clf = RandomForestClassifier(criterion=c, max_depth=i, n_jobs=-1, verbose=0)

    scores = cross_val_score(clf, trainData, awesometrain, scoring='f1_weighted', cv=10)
    output = 'criterion: %s, max_depth: %d, F1: %f, F1std: %f' % (c, i, np.average(scores), np.std(scores))
    if np.average(scores)-np.std(scores) > bestScore:
      bestOutput = output
      bestScore = np.average(scores)-np.std(scores)
    print(output)
print('The best option for random forest is: %s' % (bestOutput))

criterion: entropy, max_depth: 7, F1: 0.590241, F1std: 0.004109
criterion: entropy, max_depth: 8, F1: 0.595091, F1std: 0.004941
criterion: entropy, max_depth: 9, F1: 0.597115, F1std: 0.004398
criterion: entropy, max_depth: 10, F1: 0.599393, F1std: 0.003988
criterion: entropy, max_depth: 11, F1: 0.601197, F1std: 0.004324
criterion: entropy, max_depth: 12, F1: 0.602585, F1std: 0.004626
criterion: entropy, max_depth: 13, F1: 0.603946, F1std: 0.005421
criterion: entropy, max_depth: 14, F1: 0.603287, F1std: 0.004111
criterion: gini, max_depth: 7, F1: 0.591253, F1std: 0.003647
criterion: gini, max_depth: 8, F1: 0.595340, F1std: 0.004301
criterion: gini, max_depth: 9, F1: 0.596856, F1std: 0.003987
criterion: gini, max_depth: 10, F1: 0.598743, F1std: 0.003958


In [ ]:
# Decision Tree feature select

clf = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth = 2)
score, dictionary = RFS(clf, trainData, awesometrain)

print("best combination of features for decision tree")
for i in dictionary:
  if(abs(score - dictionary.get(i)) < 0.0001):
    print(i)
    print(dictionary.get(i))
    break;

#0.7105

In [ ]:
# Decision Tree confusion matrix + F1 score
from sklearn.metrics import confusion_matrix

x_train, x_test, y_train, y_test = train_test_split(trainData, awesometrain, test_size=0.1)
clf = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth = 2)
clf.fit(x_train, y_train)
predict = clf.predict(x_test)
print(confusion_matrix(y_test, predict))
print(f1_score(y_test, predict))
# Confusion: [[ 549 2851]
#             [ 412 3497]]
# f1: 0.6818757921419518

[[ 549 2851]
 [ 412 3497]]
0.6818757921419518


In [ ]:
#KNN Feature Selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

#initialize variables
columns = trainData.columns
scores = np.zeros(trainData.shape[1])
clf = KNeighborsClassifier(n_neighbors=100, n_jobs=-1)

featureSelection(clf, trainData, awesometrain)

In [ ]:
#KNN hyperparameter tuning
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

bestscore = 0
bestoutput = ""
for i in range(50, 151, 50):
  for j in ['uniform', 'distance']:
    for k in ['auto', 'ball_tree', 'kd_tree', 'brute']:
      for l in range(15, 46, 15):
        clf = KNeighborsClassifier(n_neighbors=i, weights = j, algorithm=k,
                                   leaf_size=l, n_jobs=-1)
        x_train, x_test, y_train, y_test = train_test_split(trainData, awesometrain, test_size=0.1)
        clf.fit(x_train, y_train)
        scores = f1_score(y_test, clf.predict(x_test))
        #scores = cross_val_score(clf, trainData, awesometrain, scoring='f1', cv=10)
        avgscore = np.average(scores)
        output = "neightbors: %d, weights: %s, algorithm: %s, leaf_size: %d, F1: %f" % (i, j, k, l, np.average(avgscore))
        if avgscore > bestscore:
          bestscore = avgscore
          bestoutput = output
        print(output)
print('The best option for KNN is: ', bestoutput)
# best option
# The best option for KNN is:  neightbors: 100, weights: distance, algorithm: auto, leaf_size: 15, F1: 0.668424

In [ ]:
# KNN Hyperparameter tuning+feature Selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

clf = KNeighborsClassifier(n_neighbors=100, weights = 'distance', algorithm='auto', leaf_size=15, n_jobs=-1)

print("Baseline: %f" % (np.average(cross_val_score(clf, trainData,
                                 awesometrain, scoring='f1', cv=10))))
print("feature_selection: %f" % (np.average(cross_val_score(clf, trainData.iloc[:, [0,2,3,4,6,8,9,11,16,17]],
                                 awesometrain, scoring='f1', cv=10))))

#

In [ ]:
#Naive Bayes Feature Selection
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

#initialize variables
columns = trainData.columns
scores = np.zeros(trainData.shape[1])
clf = BernoulliNB()

featureSelection(clf, trainData, awesometrain)

In [ ]:
#Naive Bayes hyperparameter tuning
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

bestscore = 0
bestoutput = ""
algo = [GaussianNB(), BernoulliNB(force_alpha=True)]
name = ['GaussianNB()', 'BernoulliNB()']
for i in range(2):
  #calculate score
  score = np.average(cross_val_score(algo[i], trainData, awesometrain, scoring='f1', cv=10))

  #update output
  output = "Algorithm: %s, f1 score: %.3f" % (name[i], score)

  #update best score and output
  if score > bestscore:
    bestscore = score
    bestoutput = output

  print(output)

print('The best option for NB is: ', bestoutput)
# best option for NB is BernoulliNB, f1 score: 0.600

In [ ]:
#NB feature selection + hyper parameter tuning; f1-score: 62%
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

#initialize variables
algo = [GaussianNB(), BernoulliNB(force_alpha=True)]
name = ['GaussianNB():', 'BernoulliNB():']


for i in range(2):
  x_train, x_test, y_train, y_test = train_test_split(trainData, awesometrain)
  clf = algo[i]
  clf.fit(x_train, y_train)
  predict=clf.predict(x_test)
  print(name[i])
  print("Recall: %f" % (metrics.recall_score(y_test, predict)))
  print("Precision: %f" % (metrics.precision_score(y_test, predict)))
  print("f1: %f" % (f1_score(y_test, predict)))

  '''
  clf = algo[1]
  score, visited = RFS(clf, trainData, awesometrain, {})
  print("Baseline: %s, f1-score: %f" % (name[1], np.average(cross_val_score(clf, trainData,
                                 awesometrain, scoring='f1', cv=10))))
  print("feature_selection: %s, f1-score: %f" % (name[1], score))
'''
# Baseline: GaussianNB(), f1-score: 0.354134
# feature_selection: GaussianNB(), f1-score: 0.675947
# Baseline: BernoulliNB, f1-score: f1-score: 0.597
# feature_selection: BernoulliNB: f1-score: 0.691

In [ ]:
#Random Forest Classifier
#feature selection
#create classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
clasf = RandomForestClassifier(criterion='log_loss', verbose = 2)
cv = StratifiedKFold(n_splits = 10)
f = RFECV(estimator = clasf, step = 1, cv=cv, scoring = 'f1', n_jobs =-1)
features = f.fit(trainData, awesometrain)

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
#train new random forest with tuned parameters
clasfwithhyper = RandomForestClassifier(criterion = 'entropy', max_features = 3, min_samples_leaf = 5, min_samples_split = 10)
rfnew = RFECV(clasfwithhyper, step = 1, cv = StratifiedKFold(n_splits=10), scoring = 'f1', n_jobs=  -1)
newfeat = rfnew.fit(trainData, awesometrain)

#6
from sklearn import metrics
y_predlog = newfeat.predict(trainData)
cm = metrics.confusion_matrix(awesometrain, y_predlog)
cm
f1 = np.average(cross_val_score(rfnew, trainData, awesometrain, scoring = 'f1', cv=10))
precision = np.average(cross_val_score(rfnew, trainData,awesometrain, scoring = 'precision', cv=10))
recall = np.average(cross_val_score(rfnew, trainData, awesometrain, scoring = 'recall', cv=10))
print(f1, precision, recall)
print(cm)



In [ ]:
#SVM Hyperparameter Tuning
#max_iter=-1: F1 .668
bestOutput = ""
bestScore = 0
for k in ['rbf']:
  for i in range(1):
    for mi in range(-1):
      clf = svm.SVC(kernel = 'rbf', C=1, max_iter=-1)

      scores = cross_val_score(clf, trainData, awesometrain, scoring='f1', cv=2)
      output = 'kernel: %s, C: %d, max_iter: %d, F1: %f' % (k, i, mi, np.average(scores))

      if np.average(scores) > bestScore:
        bestOutput = output
        bestScore = np.average(scores)

      print(output)
print('The best option for svm is: ', bestOutput)

In [ ]:
#Late fusion with KNN, Logistic regression, decision tree, random forst, Bernoulli Naive Bayes
# F1-score: NB: 0.694, KNN: 0.651, LogReg: 0.660, decisionTree: 0.662, randomForest: 0.663
# with trainData: NB: 0.600, KNN: 0.654, LogReg: 0.660, decisionTree: 0.663, randomForest: 0.659
# with feature selection: NB: , KNN: , Logreg: , decisionTree: , randomForest:
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn import metrics

#initialize classifiers
NB = BernoulliNB()
KNN = KNeighborsClassifier(n_neighbors=100, leaf_size=15, weights='distance', algorithm='auto')
logReg = LogisticRegressionCV(cv=3, refit = False, solver = 'saga')
decisionTree = DecisionTreeClassifier(criterion="gini", splitter='best', max_depth = 3)
randomForest = RandomForestClassifier(criterion='log_loss', min_samples_split = 5, min_samples_leaf = 2)

#create data
x_train, x_test, y_train, y_test = train_test_split(trainData,awesometrain,test_size = 0.1)

# fit classifiers to data
NB.fit(x_train, y_train)
KNN.fit(x_train, y_train)
logReg.fit(x_train, y_train)
decisionTree.fit(x_train, y_train)
randomForest.fit(x_train, y_train)

#initialize panda datafram
lateFusion = pd.DataFrame()
lateFusionTest = pd.DataFrame()

# get columns and append to pandas dataframe
lateFusion["Naive Bayes"] = NB.predict_proba(x_train)[:,1]
lateFusion["KNN"] = KNN.predict_proba(x_train)[:,1]
lateFusion["logReg"] = logReg.predict_proba(x_train)[:,1]
lateFusion["decisionTree"] = decisionTree.predict_proba(x_train)[:,1]
lateFusion["randomForest"] = randomForest.predict_proba(x_train)[:,1]

lateFusionTest["Naive Bayes"] = NB.predict_proba(x_test)[:,1]
lateFusionTest["KNN"] = KNN.predict_proba(x_test)[:,1]
lateFusionTest["logReg"] = logReg.predict_proba(x_test)[:,1]
lateFusionTest["decisionTree"] = decisionTree.predict_proba(x_test)[:,1]
lateFusionTest["randomForest"] = randomForest.predict_proba(x_test)[:,1]
# combine previous data with lateFusion for more info
#lateFusion.index = np.arange(1,len(lateFusion)+1)

# f1-score no trainingdata just late fusion
'''
clf = DecisionTreeClassifier(criterion="gini", splitter='best', max_depth = 3)
clf.fit(lateFusion, y_train)
predict = clf.predict(lateFusionTest)
f1 = f1_score(y_test, predict)
recall = metrics.recall_score(y_test, predict)
precision = metrics.precision_score(y_test, predict)
print(metrics.confusion_matrix(y_test, predict))
print(f1, precision, recall)

lateFusion = pd.concat([trainData, lateFusion], axis=1)

# run late Fusion
print("NB")
for i in range(len(lateFusion.columns)):
  print("# of features: %d f1-score: %.3f" % (i, np.average(cross_val_score(NB, lateFusion.iloc[:, featureSelection(NB, lateFusion, awesometrain)[:i+1]], awesometrain, scoring='f1', cv=10)), ))
'''

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(trainData, awesometrain, test_size = 0.35)
clf = LogisticRegressionCV(cv = 10)
cv = StratifiedKFold(n_splits=10)
rf = RFE(clf, n_features_to_select=8)
features1 = rf.fit(X_train, Y_train)
features1.support_
store = pd.DataFrame({'columns':X_train.columns, 'Kept': features1.support_})
new= X_train.iloc[:, features1.support_]

In [ ]:
from sklearn.model_selection import GridSearchCV
logreg = {'solver':['lbfgs', 'liblinear','newton-cg','sag', 'saga','newton-cholesky'], 'refit':[True, False], 'penalty':['l2', 'elasticnet']}
def gridsearch(model, params_grid):
  search = GridSearchCV(estimator = clf, param_grid = params_grid, scoring = 'f1', cv = 3, n_jobs=-1)
  return search.fit(new, Y_train)
bestrf = gridsearch(clf, logreg).best_estimator_
bestrf

LogisticRegressionCV(cv=10, refit=False)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
best = LogisticRegressionCV(solver = 'saga', cv = 10, refit = False)
best.fit(new, Y_train)

LogisticRegressionCV(cv=10, refit=False, solver='saga')

In [ ]:
from sklearn import metrics
y_predlog = best.predict(X_test.iloc[:, features1.support_])
cm = metrics.confusion_matrix(Y_test, y_predlog)

f1 = np.average(cross_val_score(best, trainData.iloc[:, features1.support_], awesometrain, scoring = 'f1', cv=10))
precision = np.average(cross_val_score(best, trainData.iloc[:, features1.support_], awesometrain, scoring = 'precision', cv=10))
recall = np.average(cross_val_score(best, trainData.iloc[:, features1.support_], awesometrain, scoring = 'recall', cv=10))
print(f1, precision, recall)
print(cm)

0.6522842591749229 0.5925076418830011 0.7255185786006113
[[5088 6968]
 [3569 9954]]
